In [ ]:
#!pip install -q langgraph langchain langchain-openai
import os, re, textwrap
from typing import Dict, List
from langgraph.graph import StateGraph
from langchain_openai import ChatOpenAI
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

from openai import OpenAI
client = OpenAI()

MODEL = "gpt-4o-mini"

# pip install -q langgraph langchain langchain-openai
import os, re, textwrap, difflib
from typing import Dict, List
from langgraph.graph import StateGraph
from langchain_openai import ChatOpenAI

# --- config ---
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or "YOUR_OPENAI_KEY"
MODEL = "gpt-4o-mini"
WRAP_WIDTH = 80

# --- LLMs per role ---
llm_researcher = ChatOpenAI(model=MODEL, temperature=0.1)
llm_writer     = ChatOpenAI(model=MODEL, temperature=0.6)
llm_reviewer   = ChatOpenAI(model=MODEL, temperature=0.2)

# --- text wrap helper ---
def wrap(text: str) -> str:
    return textwrap.fill(text.strip(), width=WRAP_WIDTH)

# --- nodes ---
#define researcher
def researcher(state: Dict) -> Dict:
    topic = state["topic"].strip()
    prompt = f"""List 4–6 precise, factual bullet points about: {topic}.
Return only bullets, each starting with '- '. No intro/outro."""
    text = llm_researcher.invoke(prompt).content

    bullets: List[str] = [
        re.sub(r"^-\\s*", "", line).strip()
        for line in text.splitlines()
        if line.strip().startswith("-")
    ] or [text.strip()]

    state["research"] = bullets
    print("\n🔎 Researcher bullets:")
    for i, b in enumerate(bullets, 1):
        print(f"  {i}. {wrap(b)}")
    return state

#define writer
def writer(state: Dict) -> Dict:
    bullets = state["research"]
    prompt = f"""Write one flowing paragraph (75–100 words) from these bullets.
Make it lively and persuasive, like a tech blogger trying to excite readers.
Feel free to speculate a bit and use vivid language.

Bullets:
{chr(10).join(f"- {b}" for b in bullets)}"""
    paragraph = llm_writer.invoke(prompt).content.strip()
    state["draft"] = paragraph

    print("\n✍️ Writer draft:\n")
    print(wrap(paragraph))
    return state

#define reviewer
def reviewer(state: Dict) -> Dict:
    draft = state["draft"]
    prompt = """Rewrite to:
- Remove hype and emotional tone
- Remove all speculation
- Shorten to ~50 words
- Keep strictly factual, neutral tone
Return ONE paragraph only."""
    reviewed = llm_reviewer.invoke([
        ("system", "You edit for clarity, accuracy, brevity, and neutrality."),
        ("human", draft),
        ("human", prompt),
    ]).content.strip()
    state["final"] = reviewed

    print("\n🧪 Reviewer final:\n")
    print(wrap(reviewed))

    return state

# --- graph ---
graph = StateGraph(dict)
graph.add_node("researcher", researcher)
graph.add_node("writer", writer)
graph.add_node("reviewer", reviewer)

graph.set_entry_point("researcher")
graph.add_edge("researcher", "writer")
graph.add_edge("writer", "reviewer")

app = graph.compile()

# --- run ---
#result = app.invoke({"topic": "Impact of artificial intelligence in education"})
result = app.invoke({"topic": "Difference between AI Single Agent vs Multi Agent"})



🔎 Researcher bullets:
  1. - AI Single Agent refers to a system where one autonomous entity operates
independently to achieve its goals, while Multi-Agent AI involves multiple
autonomous entities that interact and collaborate or compete with each other.
  2. - In Single Agent systems, decision-making is based solely on the agent's own
observations and internal state, whereas Multi-Agent systems require
communication and coordination among agents to optimize collective outcomes.
  3. - Single Agent AI typically focuses on solving problems in isolation, while
Multi-Agent AI is designed to handle complex scenarios that involve distributed
problem-solving and negotiation among agents.
  4. - The complexity of algorithms and computational resources tends to increase in
Multi-Agent systems due to the need for managing interactions and potential
conflicts between agents.
  5. - Single Agent systems are often simpler to design and implement, while Multi-
Agent systems can provide enhanced fle